<a href="https://www.kaggle.com/code/ayushgpt8/merging-data?scriptVersionId=156574569" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Merging Data

Combining and reshaping data from multiple sources

For this notebook, we will explore various techniques for combining and reshaping data from multiple sources. We will use a dataset from kaggle, found at: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data and https://www.kaggle.com/datasets/ayushgpt8/new-york-city-airbnb-temperature/data

Table of contents:
* Combining data using pandas library
* Validating Merges
* Debugging Chains
* Exporting to Excel

In [1]:
import numpy as np
import pandas as pd

While on kaggle, the dataset can be directly attached from the side panel and read from `kaggle/input`. If you are running from kaggle, then you can continue as is. if you are running it elsewhere, comment out the cell below this and uncomment the one where the data is read from `data/` directory

In [2]:
# assuming you are running on kaggle and have attached the dataset as input from the sidepanel
airbnb = pd.read_csv('/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv', dtype_backend='pyarrow', engine='pyarrow')
airbnb

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2


In [3]:
temps = pd.read_csv(
     "/kaggle/input/new-york-city-airbnb-temperature/nyc-ab-temp.csv", index_col=0, dtype_backend="pyarrow", engine="pyarrow"
)
    
temps

,lat,lon,temp
,,,
0,40.64749,-73.97237,72.0
1,40.75362,-73.98377,57.0
2,40.80902,-73.94190,76.0
3,40.68514,-73.95976,24.0
4,40.79851,-73.94399,27.0
...,...,...,...
48890,40.67853,-73.94995,76.0
48891,40.70184,-73.93317,71.0
48892,40.81475,-73.94867,44.0


In [4]:
# assuming you have downloaded the dataset and stored it in data/ directory.
# import zipfile

# with zipfile.ZipFile("data/AB_NYC_2019.csv.zip") as zip:
#     print(zip.namelist())

# airbnb = pd.read_csv("data/AB_NYC_2019.csv.zip", dtype_backend="pyarrow", engine="pyarrow")
# temps = pd.read_csv(
#     "data/nyc-ab-temp.csv", index_col=0, dtype_backend="pyarrow", engine="pyarrow"
# )

## Merging

Lets see how merging takes place with pandas

In [5]:
# simple method, but will error out
# (airbnb
#     .merge(temps)
# )

This errors out since by default, pandas look for similar columns to merge on. The error message here clearly tells us that it couldn't find any columns to perform merge on. Just for fun, lets explorer a little.

In [6]:
(airbnb
    .columns
    .intersection(temps.columns)
)

Index([], dtype='object')

As expected, there are no common columns between the two dataframes.

In [7]:
# Here are airbnb columns
airbnb.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [8]:
# Here are temps columns
temps.columns

Index(['lat', 'lon', 'temp'], dtype='object')

We will need to explicitly tell pandas the columns to merge on.

In [9]:
(airbnb
    .merge(temps, 
           left_on=['latitude', 'longitude'], 
           right_on=['lat', 'lon']
          )
)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,lat,lon,temp
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,40.64749,-73.97237,72.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,40.75362,-73.98377,57.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365,40.80902,-73.94190,76.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,40.68514,-73.95976,24.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,40.79851,-73.94399,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9,40.67853,-73.94995,76.0
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36,40.70184,-73.93317,71.0
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27,40.81475,-73.94867,44.0
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2,40.75751,-73.99112,73.0


The `left_on` parameter is refering to the airbnb dataframe and the `right_on` is refering to the temps dataframe.

## Validating Merges

There are more than 1 ways one can merge two dataframes and it is important to validate the results of the merge before performing any more operations on that dataframe.

The `how` parameter in the `df.merge()` function in pandas determines the type of merge to be performed. It corresponds to the `JOIN` operation in SQL. Here are the options:

- `'inner'`: Use intersection of keys from both dataframes (SQL: `INNER JOIN`). This is the default option.
- `'outer'`: Use union of keys from both dataframes (SQL: `FULL OUTER JOIN`).
- `'left'`: Use only keys from left dataframe (SQL: `LEFT OUTER JOIN`).
- `'right'`: Use only keys from right dataframe (SQL: `RIGHT OUTER JOIN`).
- `'cross'`: Perform a cartesian product of the two dataframes, which means it combines each row of the first dataframe with each row of the second dataframe. This is equivalent to a `CROSS JOIN` in SQL.


The `validate` argument in pandas `merge()` function is used to check whether the specified type of merge is valid (or allowed) based on the relationship between the keys in both dataframes. Here are the options:

- `'one_to_one'`: Check if merge keys are unique in both left and right datasets. If not, it will throw a MergeError. This is essentially checking if the merge is a one-to-one merge.
- `'one_to_many'`: Check if merge keys are unique in left dataset. If not, it will throw a MergeError. This is essentially checking if the merge is a one-to-many merge.
- `'many_to_one'`: Check if merge keys are unique in right dataset. If not, it will throw a MergeError. This is essentially checking if the merge is a many-to-one merge.
- `'many_to_many'`: No validation is done. This is essentially a many-to-many merge.

Let's see these in action with an example. The datasets we use can also be used but visualizing them would be difficult.

In [10]:
left = pd.DataFrame({"name": ["Ravi", "Jose", "Jose"], "pet": ["Dog", "Cat", "Dog"]})
right = pd.DataFrame({"name": ["Ravi", "Jose", "Sally"], "age": [10, 17, 5]})

In [11]:
display(left)
display(right)

,name,pet
0,Ravi,Dog
1,Jose,Cat
2,Jose,Dog


,name,age
0,Ravi,10
1,Jose,17
2,Sally,5


In [12]:
left.merge(right, how='inner')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


In [13]:
left.merge(right, how='left')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


In [14]:
left.merge(right, how='right')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17
3,Sally,NaN,5


In [15]:
left.merge(right, how='outer')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17
3,Sally,NaN,5


In [16]:
left.merge(right, how='cross')

,name_x,pet,name_y,age
0,Ravi,Dog,Ravi,10
1,Ravi,Dog,Jose,17
2,Ravi,Dog,Sally,5
3,Jose,Cat,Ravi,10
4,Jose,Cat,Jose,17
5,Jose,Cat,Sally,5
6,Jose,Dog,Ravi,10
7,Jose,Dog,Jose,17
8,Jose,Dog,Sally,5


In [17]:
# one_to_one validation will error out since the left does not contain unique merge keys
left.merge(right, how='inner', validate='many_to_one')

,name,pet,age
0,Ravi,Dog,10
1,Jose,Cat,17
2,Jose,Dog,17


Applying this to our airbnb datasets above, we see the same output when using `validate='many_to_one'`.

_PS_: We can mention `m:1`, `1:1`, `1:m` and `m:m` as well.

In [18]:
(airbnb
    .merge(temps, 
           left_on=['latitude', 'longitude'], 
           right_on=['lat', 'lon'],
           validate='m:1'
          )
)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,lat,lon,temp
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,40.64749,-73.97237,72.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,40.75362,-73.98377,57.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365,40.80902,-73.94190,76.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,40.68514,-73.95976,24.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,40.79851,-73.94399,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9,40.67853,-73.94995,76.0
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36,40.70184,-73.93317,71.0
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27,40.81475,-73.94867,44.0
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2,40.75751,-73.99112,73.0


## Cleanup columns

Let's clean up the columns in the resulting dataframe.

Before we do that, we'll also define a simple `debug()` function that would output the shape of the dataframe before and after we perform the operations

In [19]:
def debug(df, extra=""):
    print(f'{extra} {df.shape=}')
    return df

In [20]:
(airbnb
    .pipe(debug, extra="before")
    .merge(temps, left_on=["latitude", "longitude"], right_on=["lat", "lon"])
    .pipe(debug, extra="after")
    .columns
)

before df.shape=(48895, 16)
after df.shape=(48895, 19)


Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'lat', 'lon', 'temp'],
      dtype='object')

Here we see the `pipe()` function that allows us to pass in any arbitrary function to the dataframe. The first argument is the function name, second argument is the `extra` parameter we defined.


In [21]:
# Explicitly list columns
(airbnb
    .pipe(debug, extra="before")
    .merge(temps, left_on=["latitude", "longitude"], right_on=["lat", "lon"])
    .pipe(debug, extra="after")
    .loc[
        :,
        [
            "id",
            "name",
            "host_id",
            "host_name",
            "neighbourhood_group",
            "neighbourhood",
            "latitude",
            "longitude",
            "room_type",
            "price",
            "minimum_nights",
            "number_of_reviews",
            "last_review",
            "reviews_per_month",
            "calculated_host_listings_count",
            "availability_365",
            "temp",
        ],
    ]
    .pipe(debug, extra="limit cols")
)

before df.shape=(48895, 16)
after df.shape=(48895, 19)
limit cols df.shape=(48895, 17)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,temp
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,72.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,57.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,<NA>,<NA>,1,365,76.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,24.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,<NA>,<NA>,2,9,76.0
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,<NA>,<NA>,2,36,71.0
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,<NA>,<NA>,1,27,44.0
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,<NA>,<NA>,6,2,73.0


## Export to excel

In [22]:
xls_out = pd.ExcelWriter("/kaggle/working/airbnb_out.xlsx")
xl = (airbnb
    .pipe(debug, extra="before")
    .merge(temps, left_on=["latitude", "longitude"], right_on=["lat", "lon"])
    .pipe(debug, extra="after")
    .loc[
        :,
        [
            "id",
            "name",
            "host_id",
            "host_name",
            "neighbourhood_group",
            "neighbourhood",
            "latitude",
            "longitude",
            "room_type",
            "price",
            "minimum_nights",
            "number_of_reviews",
            "last_review",
            "reviews_per_month",
            "calculated_host_listings_count",
            "availability_365",
            "temp",
        ],
    ]
)

xl.to_excel(xls_out, sheet_name="all")
(xl.query('neighbourhood_group=="Brooklyn"').to_excel(xls_out, sheet_name="Brooklyn"))
xls_out.close()

before df.shape=(48895, 16)
after df.shape=(48895, 19)
